# Figure 5 Analysis Rough Version

# Table of Contents
* [Subset Gating](#Subset)
    * [Load Data](#Load)
    * [CD8 MAIT](#MAIT)
    * [dnT MAIT](#dnTMAIT)    
    * [gdT ADT](#ADT)
    * [gdT RNA](#RNA)

In [ ]:
quiet_library <- function(...) {
    suppressPackageStartupMessages(library(...))
}
quiet_library(Seurat)
quiet_library(ggplot2)
quiet_library(Matrix)
quiet_library(H5weaver)
quiet_library(dplyr)
quiet_library(viridis)
quiet_library(harmony)
quiet_library(Nebulosa)

## Subset Gating <a class="anchor" id="Subset"></a>

### Load Data <a class="anchor" id="Load"></a>

In [ ]:
cd8 <- readRDS(file = '/home/jupyter/Organized_Gating_Analysis/Seurat_Objects/cd8_subset.rds')

In [ ]:
sort(rownames(cd8[['ADT']]))

In [ ]:
cd8.batch <- SplitObject(cd8, split.by = 'batch_id')

In [ ]:
cd8.batch <- lapply(cd8.batch, function(x){
    NormalizeData(x, assay = 'ADT', normalization.method = 'CLR', margin = 2)})

In [ ]:
df_build <- function(seurat_object){
    meta <- seurat_object@meta.data
    meta$TCRValpha7.2 <- as.vector(seurat_object@assays$ADT['TCR-Valpha7.2',])
    meta$CD161 <- as.vector(seurat_object@assays$ADT['CD161',])
    meta$CD197 <- as.vector(seurat_object@assays$ADT['CD197',])
    meta$CD45RA <- as.vector(seurat_object@assays$ADT['CD45RA',])
    meta$CD27 <- as.vector(seurat_object@assays$ADT['CD27',])
    meta$imputed_CD197 <- as.vector(seurat_object@assays$x10_ADT['CD197',])
    meta$raw_CD197 <- as.vector(seurat_object@assays$ADT@counts['CD197',])
    meta$raw_CD45RA <- as.vector(seurat_object@assays$ADT@counts['CD45RA',])
    meta$raw_CD27 <- as.vector(seurat_object@assays$ADT@counts['CD27',])
    return(meta)}

In [ ]:
dnT <- readRDS(file = '/home/jupyter/Organized_Gating_Analysis/Seurat_Objects/dnT_subset.rds')

In [ ]:
dnT

In [ ]:
dnT.batch <- SplitObject(dnT, split.by = 'batch_id')

In [ ]:
dnT.batch <- lapply(dnT.batch, function(x){
    NormalizeData(x, assay = 'ADT', normalization.method = 'CLR', margin = 2)})

### CD8 MAIT Cells  <a class="anchor" id="MAIT"></a>

In [ ]:
options(repr.plot.width = 24, repr.plot.height = 8)
p1 <- FeatureScatter(object = cd8.batch$B065, feature1 = 'adt_TCR-Valpha7.2', feature2 = 'adt_CD161', group.by = 'predicted.t_celltype.l2') & geom_vline(xintercept = 0.75) & geom_hline(yintercept = 1.5)
p2 <- FeatureScatter(object = cd8.batch$B069, feature1 = 'adt_TCR-Valpha7.2', feature2 = 'adt_CD161', group.by = 'predicted.t_celltype.l2') & geom_vline(xintercept = 0.75) & geom_hline(yintercept = 1.25)
p3 <- FeatureScatter(object = cd8.batch$B076, feature1 = 'adt_TCR-Valpha7.2', feature2 = 'adt_CD161', group.by = 'predicted.t_celltype.l2') & geom_vline(xintercept = 0.5) & geom_hline(yintercept = 0.75)
p1 + p2 + p3

In [ ]:
mait_b065 <- subset(cd8.batch$`B065`, subset = `adt_TCR-Valpha7.2` > 0.75 & adt_CD161 > 1.5)
mait_b069 <- subset(cd8.batch$`B069`, subset = `adt_TCR-Valpha7.2` > 0.75 & adt_CD161 > 1.25)
mait_b076 <- subset(cd8.batch$`B076`, subset = `adt_TCR-Valpha7.2` > 0.5 & adt_CD161 > 0.75)

In [ ]:
mait_merge <- merge(mait_b065, c(mait_b069, mait_b076))

In [ ]:
mait_merge$mnp_analysis <- 'CD8 MAIT'

### dnT MAIT Cells  <a class="anchor" id="dnTMAIT"></a>

In [ ]:
options(repr.plot.width = 24, repr.plot.height = 8)
p1 <- FeatureScatter(object = dnT.batch$B065, feature1 = 'adt_TCR-Valpha7.2', feature2 = 'adt_CD161', group.by = 'predicted.t_celltype.l2') & geom_vline(xintercept = 0.75) & geom_hline(yintercept = 1.5)
p2 <- FeatureScatter(object = dnT.batch$B069, feature1 = 'adt_TCR-Valpha7.2', feature2 = 'adt_CD161', group.by = 'predicted.t_celltype.l2') & geom_vline(xintercept = 0.75) & geom_hline(yintercept = 1.25)
p3 <- FeatureScatter(object = dnT.batch$B076, feature1 = 'adt_TCR-Valpha7.2', feature2 = 'adt_CD161', group.by = 'predicted.t_celltype.l2') & geom_vline(xintercept = 0.5) & geom_hline(yintercept = 0.75)
p1 + p2 + p3

In [ ]:
dnTmait_b065 <- subset(dnT.batch$`B065`, subset = `adt_TCR-Valpha7.2` > 0.75 & adt_CD161 > 1.5)
dnTmait_b069 <- subset(dnT.batch$`B069`, subset = `adt_TCR-Valpha7.2` > 0.75 & adt_CD161 > 1.25)
dnTmait_b076 <- subset(dnT.batch$`B076`, subset = `adt_TCR-Valpha7.2` > 0.5 & adt_CD161 > 0.75)

In [ ]:
dntmait_merge <- merge(dnTmait_b065, c(dnTmait_b069, dnTmait_b076))

In [ ]:
dntmait_merge$mnp_analysis <- 'dnT MAIT'

In [ ]:
mait_merge <- merge(mait_merge, dntmait_merge)

In [ ]:
saveRDS(mait_merge, 'mait_merge.rds')

In [ ]:
dnT_b065 <- subset(dnT.batch$`B065`, subset = `adt_TCR-Valpha7.2` > 0.75 & adt_CD161 > 1.5, invert = TRUE)
dnT_b069 <- subset(dnT.batch$`B069`, subset = `adt_TCR-Valpha7.2` > 0.75 & adt_CD161 > 1.25, invert = TRUE)
dnT_b076 <- subset(dnT.batch$`B076`, subset = `adt_TCR-Valpha7.2` > 0.5 & adt_CD161 > 0.75, invert = TRUE)

In [ ]:
dnt_nonMAIT <- merge(dnT_b065,c(dnT_b069,dnT_b076))

In [ ]:
saveRDS(dnt_nonMAIT, 'nonMAIT_merge.rds')

In [ ]:
sessionInfo()